In [2]:
import tensorflow as tf
from tensorflow import keras
from tabulate import tabulate
import pandas as pd
import numpy as np
import urllib
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


import sys
print(sys.version)

def getApiKey(filename):
    api_key_file = open(filename, 'r')
    return api_key_file.read().rstrip()

def standardizeTuple(t, d):
    t -= np.mean(d, axis=0)
    t /= np.std(d, axis=0)
    return t

def standardize(a):
    scaler = StandardScaler()
    a = scaler.fit_transform(a)
    d = np.array(a)
    d = d.astype('float32')
    d -= np.mean(d, axis=0)
    d /= np.std(d, axis=0)
    return np.array(d)

def getOneVideoStats(video_id, api_key):
    searchUrl="https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&key="+api_key+"&part=statistics,snippet,content_details"
    response = urllib.request.urlopen(searchUrl).read()
    data = json.loads(response.decode('utf-8'))
    try:
        viewCount = data['items'][0]['statistics']['viewCount']
        likeCount = data['items'][0]['statistics']['likeCount']
        dislikeCount = data['items'][0]['statistics']['dislikeCount']
        commentCount = data['items'][0]['statistics']['commentCount']
        categoryId = data['items'][0]['snippet']['categoryId']
        tagLength = len(data['items'][0]['snippet']['tags'])
        likeRatioCount = int(likeCount)/(int(likeCount)+int(dislikeCount))
        commentRatioCount = int(commentCount)/int(viewCount)
        return [[viewCount,likeCount,dislikeCount,commentCount, categoryId, tagLength]]
    except (KeyError, IndexError):
        return

def main():
    trendingSet = pd.read_csv('../data/new-datasets/trending_dataset.csv')
    nontrendingSet = pd.read_csv('../data/new-datasets/nontrending_dataset.csv')
    trendingSet = trendingSet[['views', 'likes', 'dislikes', 'comments', 'category_id', 'tags_length']]
    nontrendingSet = nontrendingSet[['views', 'likes', 'dislikes', 'comments', 'category_id', 'tags_length']]
    #trendingSet['likes_ratio'] = trendingSet['likes']/(trendingSet['likes']+trendingSet['dislikes'])
    #trendingSet['comments_ratio'] = trendingSet['comments']/(trendingSet['views'])
    
    #nontrendingSet['likes_ratio'] = nontrendingSet['likes']/(nontrendingSet['likes']+nontrendingSet['dislikes'])
    #nontrendingSet['comments_ratio'] = nontrendingSet['comments']/(nontrendingSet['views'])
    
    
    #print(trendingSet)
    trending_stats_non = trendingSet.values
    nontrending_stats_non = nontrendingSet.values
    #print(trending_stats_non.shape)
    #print(nontrending_stats_non)
    
    #nontrending_stats_non = np.load('nontrending_stats.npy')
    #trending_stats_non = np.load('trending_stats.npy')
    # combine trending and nontrending data, then standardize them
    all_data = np.concatenate((trending_stats_non[0:6000,:], nontrending_stats_non[0:6000]), axis=0)
    
    #all_data = all_data.astype('float32')
    #print(all_data.shape)
    standardized_data = standardize(all_data)
    # sample tuple for a random video (views, likes, dislikes, commentCount)
    exampleTuple = [[1713501,84894,2855,15155, 20, 12]]
    print(all_data.shape)
    exampleTuple = standardizeTuple(exampleTuple, all_data)
    # split em up
    trending_stats = standardized_data[0:6000,:]
    nontrending_stats = standardized_data[6001:,:]
    # curate sets of data necessary for neural net
    train_data = np.concatenate((trending_stats[0:2900,:], nontrending_stats[0:2900,:]), axis = 0)
    test_data = np.concatenate((trending_stats[2901:5801,:], nontrending_stats[2901:5801,:]), axis = 0)
    train_labels = np.concatenate((np.ones(2900), np.zeros(2900)), axis = 0)
    test_labels = np.concatenate((np.ones(2900), np.zeros(2900)), axis = 0)


    model = keras.Sequential([
        keras.layers.Dense(512, input_dim=6,activation=tf.nn.relu),
        keras.layers.Dense(512, activation=tf.nn.relu),
        (keras.layers.Dropout(0.50)),
        keras.layers.Dense(512, activation=tf.nn.relu),
        (keras.layers.Dropout(0.50)),
        keras.layers.Dense(2, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.train.AdamOptimizer(),
                    loss ='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    
    #train_data = train_data.sample(frac=1)
    model.fit(train_data, train_labels, epochs=50)
    predictions = model.predict(test_data)
    #print(predictions)
    #print(test_labels.shape)
    #truth = []
    #print(test_labels)
    #for i in range(len(predictions)):
    #    if(predictions[i][0]==1):
    #        truth.append(0)
    #    else:
    #        truth.append(1)
    #print(predictions.shape)
    #cm = tf.contrib.metrics.confusion_matrix(test_labels, truth)
    #print(cm)
    #return 
    
    test_loss, test_acc = model.evaluate(test_data, test_labels)

    print("\n\nTest Set Accuracy:" + str(test_acc))

    #history = model.fit(X, Y, validation_split=0.33, epochs=1000, batch_size=10, verbose=0, callbacks=[tb, early_stop])
    # extra code so you can test individual video ids and see if trending or nontrending
    apiKey = getApiKey('apikey.txt')
    videoId = ''
    while videoId != 'quit':
        print("Predict a video by entering a video id (or type 'quit' to exit): ")
        videoId = input()
        video = np.array(getOneVideoStats(videoId, apiKey), dtype='|S10').astype(float)
        # print table of video data
        print("\n" + tabulate(video, headers=['ViewCount', 'LikeCount', 'DislikeCount', 'CommentCount', 'CategoryId', 'Tags']))
        video = standardizeTuple(video, all_data)
        # if video data was empty dont do a prediction
        if (len(video) != 0):
            print(video)
            prediction = model.predict(video)
            # print table of prediction probabilities
            print("\n" + tabulate(prediction, headers=['P(Non-Trending)', 'P(Trending)']))
            # print prediction for video
            if (np.argmax(prediction) == 1):
                print("\nPredicted Trending Video\n")
            else:
                print("\nPredicted Non-Trending Video\n")
        else:
            print("Video missing views, likes, dislikes, or commentCount...")


main()

3.5.6 |Anaconda custom (64-bit)| (default, Aug 26 2018, 16:05:27) [MSC v.1900 64 bit (AMD64)]


C:\Users\John\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(12000, 6)
Epoch 1/50
5800/5800 [==============================] - 2s 417us/step - loss: 0.5414 - acc: 0.7172 1s - loss: 0.5710
Epoch 2/50
5800/5800 [==============================] - 2s 296us/step - loss: 0.4661 - acc: 0.7816 0s - loss: 0.4684 - acc: 0.78
Epoch 3/50
5800/5800 [==============================] - 2s 343us/step - loss: 0.4220 - acc: 0.8028
Epoch 4/50
5800/5800 [==============================] - 2s 330us/step - loss: 0.4025 - acc: 0.8217
Epoch 5/50
5800/5800 [==============================] - 2s 287us/step - loss: 0.3733 - acc: 0.8410
Epoch 6/50
5800/5800 [==============================] - 2s 368us/step - loss: 0.3742 - acc: 0.8350
Epoch 7/50
5800/5800 [==============================] - 2s 389us/step - loss: 0.3633 - acc: 0.8414
Epoch 8/50
5800/5800 [==============================] - 2s 415us/step - loss: 0.3512 - acc: 0.8519 1s - loss: 0.
Epoch 9/50
5800/5800 [==============================] - 2s 314us/step - loss: 0.3502 - acc: 0.8547
Epoch 10/50
5800/5800 [=============

TypeError: the JSON object must be str, not 'bytes'